In [1]:
import main
from os import listdir
import pandas as pd
import TAtT
import Christofides
import time as T
import tracemalloc

In [2]:

df = pd.DataFrame({'Nome':['a'], 'Tamanho':[0], 'BnB':[0], 'timeBnB':[0], 'spaceBnB':[0], 'TAtT':[0], 'timeTAtT':[0], 'spaceTAtT':[0], 'Christofides':[0], 'timeChristofides':[0], 'spaceChristofides':[0]})

In [ ]:
files = list(listdir('ALL_tsp'))
dfTAtT = pd.DataFrame({'Name':['a'],'Val':[0],'time':[0],'space':[0]})
for file in files:
    name = file
    (G,_,_) = main.Graph('ALL_tsp/'+file)
    init = T.time()
    tracemalloc.start()
    snapshot1 = tracemalloc.take_snapshot()
    (_, W) = TAtT.TatT(G,1)
    snapshot2 = tracemalloc.take_snapshot()
    diferenca = snapshot2.compare_to(snapshot1, 'lineno')
    space = round(sum(stat.size_diff for stat in diferenca), 2)
    time = T.time() - init
    newrow = pd.DataFrame({'Name':[file],'Val':[W],'time':[round(time,2)],'space':[space]})
    dfTAtT = pd.concat([dfTAtT,newrow],ignore_index=True)
    dfTAtT.to_csv('CSV/TAtT.csv')
    print('add ',name,W,time,space)
dfTAtT.drop([0],axis=0)


In [ ]:
dfChristofides = pd.DataFrame({'Name':'a','Val':0,'time':0,'space':[0]})
for file in files:
    name = file
    G = main.Graph(file)
    init = time.time()
    tracemalloc.start()
    snapshot1 = tracemalloc.take_snapshot()
    (_, W) = Christofides.Christofides(G,1)
    snapshot2 = tracemalloc.take_snapshot()
    space = snapshot2.compare_to(snapshot1, 'lineno')
    time = T.time() - init
    newrow = pd.DataFrame({'Name':[file],'Val':[W],'time':[time],'space':[space]})
    dfChristofides = pd.concat([dfChristofides,newrow],ignore_index=True)
    dfChristofides.to_csv('CSV/Christofides.csv')
    print('add ',name,W,time,space)
dfChristofides.drop([0],axis=0)